In [21]:
#import packages
import os
import torch
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader, random_split
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from skimage import io, transform
import numpy as np

Load image data using pytorch data loader, including process such as horizontal flip and normalization

In [16]:
def get_transforms(mean, std):
    
    list_transforms = []
    list_transforms.extend([HorizontalFlip(p=0.5)])
    list_transforms.extend([Normalize(mean=mean, std=std, p=1), ToTensorV2()])
    list_trfms = Compose(list_transforms)
    return list_trfms

In [29]:
class Nuclie_data(Dataset): # inherit from Dataset class provided by PyTorch
    def __init__(self, path):
        self.path = path
        self.folders = os.listdir(os.path.join(path, 'images/stage1_train'))
        self.transforms = get_transforms(0.5, 0.5)

    def __len__(self):
        return len(self.folders)

    def __getitem__(self, idx):
        image_folder = os.path.join(self.path, 'images/stage1_train/', self.folders[idx], 'images/')
        mask_folder = os.path.join(self.path, 'images/stage1_train/', self.folders[idx], 'masks/')
        image_path = os.path.join(image_folder, os.listdir(image_folder)[0])

        img = io.imread(image_path)[:, :, :3].astype('float32') # only first three channel, RGB
        img = transform.resize(img,(128, 128))

        mask = self.get_mask(mask_folder, 128, 128).astype('float32')

        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        
        # check if mask is 2D
        if mask.ndim == 2:
            mask = np.expand_dims(mask, axis=0)

        mask = mask.permute(0, 2, 1)

        return (img, mask)

    def get_mask(self, mask_folder, IMG_HEIGHT, IMG_WIDTH):
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
        for mask_ in os.listdir(mask_folder):
            mask_ = io.imread(os.path.join(mask_folder, mask_))
            mask_ = transform.resize(mask_, (IMG_HEIGHT, IMG_WIDTH))
            mask_ = np.expand_dims(mask_, axis=-1)
            mask = np.maximum(mask, mask_)

        return mask   

In [30]:
base_dir = "./"
data = Nuclie_data(base_dir)

In [31]:
print(data.__len__())
print(data.__getitem__(0))

670
(tensor([[[-0.3354, -0.1203,  0.1596,  ..., -0.8039, -0.7829, -0.6807],
         [-0.4754, -0.1831,  0.0132,  ..., -0.8172, -0.8051, -0.7579],
         [-0.6319, -0.4648, -0.3931,  ..., -0.8217, -0.8146, -0.7960],
         ...,
         [-0.2834, -0.2742, -0.3361,  ..., -0.8278, -0.7971, -0.7516],
         [-0.4539, -0.4800, -0.5659,  ..., -0.8424, -0.8290, -0.8142],
         [-0.5526, -0.6608, -0.7103,  ..., -0.8492, -0.8434, -0.8378]],

        [[-0.3354, -0.1203,  0.1596,  ..., -0.8039, -0.7829, -0.6807],
         [-0.4754, -0.1831,  0.0132,  ..., -0.8172, -0.8051, -0.7579],
         [-0.6319, -0.4648, -0.3931,  ..., -0.8217, -0.8146, -0.7960],
         ...,
         [-0.2834, -0.2742, -0.3361,  ..., -0.8278, -0.7971, -0.7516],
         [-0.4539, -0.4800, -0.5659,  ..., -0.8424, -0.8290, -0.8142],
         [-0.5526, -0.6608, -0.7103,  ..., -0.8492, -0.8434, -0.8378]],

        [[-0.3354, -0.1203,  0.1596,  ..., -0.8039, -0.7829, -0.6807],
         [-0.4754, -0.1831,  0.0132,  ..